In [62]:
class Canvas:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        # Empty canvas is a matrix with element being the "space" character
        self.data = [[' '] * width for i in range(height)]

    def set_pixel(self, row, col, char='*'):
        self.data[row][col] = char

    def get_pixel(self, row, col):
        return self.data[row][col]
    
    def clear_canvas(self):
        self.data = [[' '] * self.width for i in range(self.height)]
    
    def v_line(self, row, col, w, **kargs):
        for i in range(row,row+w):
            self.set_pixel(i,col, **kargs)

    def h_line(self, row, col, h, **kargs):
        for i in range(col,col+h):
            self.set_pixel(row,i, **kargs)
            
    def line(self, x1, y1, x2, y2, char='*', **kargs):
        dx = abs(x2 - x1)
        dy = abs(y2 - y1)
        sx = 1 if x1 < x2 else -1 
        sy = 1 if y1 < y2 else -1  
        err = dx - dy  
        
        while True:
            self.set_pixel(x1, y1, char='*')  
            if x1 == x2 and y1 == y2:  
                break
            e2 = 2 * err
            if e2 > -dy:
                err -= dy
                x1 += sx  
            if e2 < dx:
                err += dx
                y1 += sy  

    def display(self):
        print("\n".join(["".join(row) for row in self.data]))

class Shape:
    def __init__(self):
        pass

    #Virtual methods
    def calc_area(self):
        raise NotImplementedError

    def calc_perimeter(self):
        raise NotImplementedError

    def get_xy_coord(self):
        raise NotImplementedError

    def get_perimeter_points(self):
        raise NotImplementedError

    def is_inside(self):
        raise NotImplementedError

    #Check for overlap with another object
    def check_overlap(self, other):

        #Get 16 xy-pair of shape
        perimeter_points=self.get_perimeter_points()

        #Check if perimeter points are inside other shape
        for x, y in perimeter_points:
            if other.is_inside(x,y):
                return True

        #Check if the other shape is inside
        other_points=other.get_perimeter_points()

        for x, y in other_points:
            if self.is_inside(x,y):
                return True

        #No overlap
        return False 

    def paint(self, canvas): pass
                     
class Rectangle(Shape):
    def __init__(self, length, width, x_coord, y_coord): 
        Shape.__init__(self) 
        self.__length=length 
        self.__width=width
        self.__x_coord=x_coord 
        self.__y_coord=y_coord

    #Calculate area
    def calc_area(self):
        return self.__length*self.__width

    #Calculate perimeter
    def calc_perimeter(self):
        return 2*(self.__length+self.__width)

    #Accessors
    def get_length(self):
        return self.__length

    def get_width(self):
        return self.__width

    def get_xy_coord(self):
        return (self.__x_coord, self.__y_coord)

    def get_perimeter_points(self):
        points=[] #List for x,y pairs
        
        for i in range(16):
            
            if i < 4:
                #Increment by fraction of the length each iteration
                x = self.__x_coord + i * (self.__length / 4)
                y = self.__y_coord #Stays constant
            
            elif i < 8:
                x = self.__x_coord + self.__length 
                
                #Increment by fraction of the width each iteration
                y = self.__y_coord + (i - 4) * (self.__width / 4)

            elif i < 12:
                #Decrement by fraction of length each iteration (right to left)
                x = self.__x_coord + self.__length - (i - 8) * (self.__length / 4)
                y = self.__y_coord + self.__width

            else:
                x = self.__x_coord
                y = self.__y_coord + self.__width - (i - 12) * (self.__width / 4)
            points.append((x, y))
        return points

    def is_inside(self, x, y):
        #Must be between left and right edges, and top and bottom
        return (self.__x_coord <= x <= self.__x_coord + self.__length and 
                self.__y_coord <= y <= self.__y_coord + self.__width)

    def paint(self, canvas):################
        canvas.v_line(self.__x_coord, self.__y_coord, self.__width)
        canvas.v_line(self.__x_coord, self.__y_coord + self.__length, self.__width)
        canvas.h_line(self.__x_coord, self.__y_coord, self.__length)
        canvas.h_line(self.__x_coord + self.__length, self.__y_coord, self.__length)

   

    def paint_rectangle_right(self, canvas):
        #Row is y
        canvas.v_line(self.__y_coord, self.__x_coord, self.__width)  # Left vertical edge
        canvas.v_line(self.__y_coord, self.__x_coord + self.__length - 1, self.__width)  # Right vertical edge
    
        #Col is x
        canvas.h_line(self.__y_coord, self.__x_coord, self.__length)  # Top horizontal edge
        canvas.h_line(self.__y_coord + self.__width - 1, self.__x_coord, self.__length)  # Bottom horizontal edge

               
class Circle(Shape):
    def __init__(self, radius, x_coord, y_coord): 
        Shape.__init__(self)
        self.__radius=radius
        self.__x_coord=x_coord 
        self.__y_coord=y_coord

    #Calculate area
    def calc_area(self):
        return math.pi*(self.__radius**2)

    #Calculate circumference
    def calc_perimeter(self):
        return 2*math.pi*self.__radius

    #Accessors
    def get_radius(self):
        return self.__radius

    def get_xy_coord(self):
        return (self.__x_coord, self.__y_coord)

    def get_perimeter_points(self):
        points = []
        for i in range(16):
            angle = 2 * math.pi * i / 16
            
            #xy=rcos(theta), rsin(theta)
            x = self.__x_coord + self.__radius * math.cos(angle)
            y = self.__y_coord + self.__radius * math.sin(angle)
            points.append((x, y))
        return points

    def is_inside(self, x, y):
        #Use distance formula
        return (x - self.__x_coord) ** 2 + (y - self.__y_coord) ** 2 <= self.__radius ** 2


class Triangle(Shape):
    def __init__(self, side_a, side_b, side_c, x_coord, y_coord): 

        #Make sure it's a valid triangle using triangle inequality theorem
        if not (side_a + side_b > side_c and side_a + side_c > side_b and side_b + side_c > side_a):
            raise ValueError("Not a valid triangle")
            
        Shape.__init__(self)
        self.__side_a=side_a
        self.__side_b=side_b 
        self.__side_c=side_c
        self.__x_coord=x_coord 
        self.__y_coord=y_coord

    def calc_area(self):
        #Heron's formula
        s=(self.__side_a+self.__side_b+self.__side_c)/2
        a=self.__side_a
        b=self.__side_b
        c=self.__side_c
        return math.sqrt(s*(s-a)*(s-b)*(s-c))

    def calc_perimeter(self):
        return self.__side_a+self.__side_b+self.__side_c

    def get_sides(self):
        return self.__side_a, self.__side_b, self.__side_c

    def get_xy_coord(self):
        return (self.__x_coord, self.__y_coord)

    def get_perimeter_points(self):
        points = []
        segments=16
        
        #(x1, y1) to (x2, y2)
        for i in range(segments // 3):
            x = self.__x_coord + i * (self.__side_a / (segments // 3))
            y = self.__y_coord
            points.append((x, y))
            
        #(x2, y2) to (x3, y3)
        for i in range(segments // 3):
            x = self.__x_coord + self.__side_a
            y = self.__y_coord + i * (self.__side_b / (segments // 3))
            points.append((x, y))
            
        #(x3, y3) to (x1, y1)
        for i in range(segments // 3):
            x = self.__x_coord + self.__side_a - i * (self.__side_c / (segments // 3))
            y = self.__y_coord + self.__side_b
            points.append((x, y))
        return points

    def is_inside(self, x, y):
        def tri_area(x1, y1, x2, y2, x3, y3):
            return abs((x1*(y2-y3) + x2*(y3-y1) + x3*(y1-y2)) / 2)
            
        A = self.calc_area()
        height = (2 * A) / self.__side_a  
    
        x1, y1 = self.__x_coord, self.__y_coord #First vertex
        x2, y2 = self.__x_coord + self.__side_a, self.__y_coord #Second vertex
        x3, y3 = self.__x_coord + (self.__side_a / 2), self.__y_coord + height #Third vertex
    
        A1 = tri_area(x, y, x2, y2, x3, y3)
        A2 = tri_area(x1, y1, x, y, x3, y3)
        A3 = tri_area(x1, y1, x2, y2, x, y)
    
        return abs(A - (A1 + A2 + A3)) < 1e-6


class CompoundShape(Shape):
    def __init__(self, shapes):
        Shape.__init__(self)
        self.shapes = shapes

    def calc_area(self):
        return sum(shape.calc_area() for shape in self.shapes)

    def calc_perimeter(self):
        return sum(shape.calc_perimeter() for shape in self.shapes)

    def get_xy_coord(self):
        return [shape.get_xy_coord() for shape in self.shapes]

    def get_perimeter_points(self):

    def is_inside(self, x, y):

    def paint(self, canvas):
        for s in self.shapes:
            s.paint(canvas)

IndentationError: expected an indented block after function definition on line 290 (3651172890.py, line 292)

In [58]:
canv_1=Canvas(50,40)
rect_1=Rectangle(15,7,5,5)
rect_1.paint_rectangle_right(canv_1)
canv_1.display()

                                                  
                                                  
                                                  
                                                  
                                                  
     ***************                              
     *             *                              
     *             *                              
     *             *                              
     *             *                              
     *             *                              
     ***************                              
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                               